# Обработка датасета

## Библиотеки и скрипты

In [1]:
import numpy as np
import pandas as pd
import re
from transliterate import translit, get_available_language_codes

# Отключаем предупреждения
import warnings
warnings.simplefilter('ignore')

In [2]:
# Функция приведения типов
def astype_col(df, colgroup, coltype):
    for colname in colgroup:
        df[colname] = df[colname].astype(coltype)
    return df

# Функция удаления столбцов
def drop_columns(df, columns):
    for column in columns:
        df = df.drop(column, axis=1)  
    return df

# Функция транслитерации
def get_translite(columns):
    lst = []
    prefix = 'YM_'
    for column in columns:
        r = translit(prefix + column, 'ru', reversed=True).lower()
        r = re.sub("\'|\(|\)|:", "", r)
        r = re.sub("\ |-|‑", "_", r)
        lst.append(r)
    return lst

# Функция получение имен столбцов с индексом
def get_idx_columns(columns):
    lst = []
    for index, value in enumerate(columns):
        print(index, value)
        lst.append(index)
    print(lst)

## Обзор данных

In [3]:
# Загружаем датасеты без 1-й строки и делаем переиндексацию с удалением столбца index
df1 = pd.read_csv('../data/hubex.ru-ClientID-Sources.csv')[1:].reset_index(drop=True) # Источники
df2 = pd.read_csv('../data/hubex.ru-ClientID-Behavior.csv')[1:].reset_index(drop=True) # Поведение
# Удаляем дубликаты по ClientID
df1 = df1.drop_duplicates(subset='ClientID', keep="first")
df2 = df2.drop_duplicates(subset='ClientID', keep="first")

In [4]:
df1.head(5)

,ClientID,Первый источник трафика,Первый переход с сайтов,Первая поисковая система,UTM Source,UTM Content,Город,Визиты,Посетители,Отказы,Глубина просмотра,Время на сайте,Новые посетители,Дней между визитами,Вернувшиеся: 1 день,Вернувшиеся: 2‑7 дней,Достижения любой цели
0,1544792334967355948,Прямые заходы,Не определено,Не определено,Не определено,Не определено,Санкт-Петербург,484,1,0.146694,2.710744,00:05:08,0,0.000000,0.659091,0.338843,780
1,1599476927749455055,Переходы по ссылкам на сайтах,hubex.cloud,Не определено,Не определено,Не определено,Санкт-Петербург,240,1,0.087500,3.791667,00:08:02,1,0.376569,0.566667,0.425000,1169
2,1571055035849817227,Прямые заходы,Не определено,Не определено,Не определено,Не определено,Санкт-Петербург,221,1,0.122172,2.927602,00:07:11,0,0.000000,0.547511,0.443439,291
3,1596550694967369930,Переходы из поисковых систем,Не определено,Google,Не определено,Не определено,Санкт-Петербург,216,1,0.143519,2.268519,00:03:11,1,1.004651,0.472222,0.504630,599
4,1589375617652616197,Прямые заходы,Не определено,Не определено,Не определено,Не определено,Санкт-Петербург,169,1,0.118343,2.207101,00:06:07,1,0.761905,0.532544,0.426036,324


## Обработка данных

In [5]:
# Производим слияние датасетов
df = pd.merge(df1, df2, how='outer')

In [6]:
df.head(5)

,ClientID,Первый источник трафика,Первый переход с сайтов,Первая поисковая система,UTM Source,UTM Content,Город,Визиты,Посетители,Отказы,...,Дней между визитами,Вернувшиеся: 1 день,Вернувшиеся: 2‑7 дней,Достижения любой цели,Дата первого визита,Дата визита,Достигнутая цель,Путь (полный) страницы входа,Путь (полный) страницы выхода,Мобильность
0,1544792334967355948,Прямые заходы,Не определено,Не определено,Не определено,Не определено,Санкт-Петербург,484,1.0,0.146694,...,0.000000,0.659091,0.338843,780.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1599476927749455055,Переходы по ссылкам на сайтах,hubex.cloud,Не определено,Не определено,Не определено,Санкт-Петербург,240,1.0,0.087500,...,0.376569,0.566667,0.425000,1169.0,NaN,NaN,NaN,NaN,NaN,NaN
2,1571055035849817227,Прямые заходы,Не определено,Не определено,Не определено,Не определено,Санкт-Петербург,221,1.0,0.122172,...,0.000000,0.547511,0.443439,291.0,NaN,NaN,NaN,NaN,NaN,NaN
3,1596550694967369930,Переходы из поисковых систем,Не определено,Google,Не определено,Не определено,Санкт-Петербург,216,1.0,0.143519,...,1.004651,0.472222,0.504630,599.0,NaN,NaN,NaN,NaN,NaN,NaN
4,1589375617652616197,Прямые заходы,Не определено,Не определено,Не определено,Не определено,Санкт-Петербург,169,1.0,0.118343,...,0.761905,0.532544,0.426036,324.0,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df.columns

Index(['ClientID', 'Первый источник трафика', 'Первый переход с сайтов',
       'Первая поисковая система', 'UTM Source', 'UTM Content', 'Город',
       'Визиты', 'Посетители', 'Отказы', 'Глубина просмотра', 'Время на сайте',
       'Новые посетители', 'Дней между визитами', 'Вернувшиеся: 1 день',
       'Вернувшиеся: 2‑7 дней', 'Достижения любой цели', 'Дата первого визита',
       'Дата визита', 'Достигнутая цель', 'Путь (полный) страницы входа',
       'Путь (полный) страницы выхода', 'Мобильность'],
      dtype='object')

In [8]:
# Удаление ненужных данных
columns = ['Посетители']
df = drop_columns(df, columns)

# Удаление "лишних" пользователей
df.loc[df['Дата визита'].isnull(), 'Дата визита'] = 'undefined'
df = df[df['Дата визита'].str.contains('202.*')]
df = df.loc[(df['Визиты'] <= 35) & (df['Достижения любой цели'] <= 100)]
df = df.loc[(df['ClientID'] != '')]

df.head(2)

,ClientID,Первый источник трафика,Первый переход с сайтов,Первая поисковая система,UTM Source,UTM Content,Город,Визиты,Отказы,Глубина просмотра,...,Дней между визитами,Вернувшиеся: 1 день,Вернувшиеся: 2‑7 дней,Достижения любой цели,Дата первого визита,Дата визита,Достигнутая цель,Путь (полный) страницы входа,Путь (полный) страницы выхода,Мобильность
130,1595636421722944149,Переходы по рекламе,Не определено,Не определено,Не определено,Не определено,Нур-Султан,14,0.571429,1.928571,...,0.0,0.928571,0.0,0.0,2020-07-25,2020-07-25,Не определено,/#secondPage,/#secondPage,1.0
291,159789595254507935,Переходы по рекламе,Не определено,Не определено,Не определено,Не определено,Не определено,9,0.222222,1.555556,...,0.0,0.888889,0.0,0.0,2020-08-20,2020-08-20,Не определено,/,/,1.0


In [9]:
# Переименование столбцов
df.columns = get_translite(df.columns)
df.columns

Index(['ym_clientid', 'ym_pervyj_istochnik_trafika',
       'ym_pervyj_perehod_s_sajtov', 'ym_pervaja_poiskovaja_sistema',
       'ym_utm_source', 'ym_utm_content', 'ym_gorod', 'ym_vizity', 'ym_otkazy',
       'ym_glubina_prosmotra', 'ym_vremja_na_sajte', 'ym_novye_posetiteli',
       'ym_dnej_mezhdu_vizitami', 'ym_vernuvshiesja_1_den',
       'ym_vernuvshiesja_2_7_dnej', 'ym_dostizhenija_ljuboj_tseli',
       'ym_data_pervogo_vizita', 'ym_data_vizita', 'ym_dostignutaja_tsel',
       'ym_put_polnyj_stranitsy_vhoda', 'ym_put_polnyj_stranitsy_vyhoda',
       'ym_mobilnost'],
      dtype='object')

**Обработка пропусков**

In [10]:
df.info()
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27396 entries, 130 to 44624
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ym_clientid                     27396 non-null  object 
 1   ym_pervyj_istochnik_trafika     27396 non-null  object 
 2   ym_pervyj_perehod_s_sajtov      27396 non-null  object 
 3   ym_pervaja_poiskovaja_sistema   27396 non-null  object 
 4   ym_utm_source                   27396 non-null  object 
 5   ym_utm_content                  27396 non-null  object 
 6   ym_gorod                        27396 non-null  object 
 7   ym_vizity                       27396 non-null  int64  
 8   ym_otkazy                       27396 non-null  float64
 9   ym_glubina_prosmotra            27396 non-null  float64
 10  ym_vremja_na_sajte              27396 non-null  object 
 11  ym_novye_posetiteli             27396 non-null  float64
 12  ym_dnej_mezhdu_vizitami       

ym_clientid                           0
ym_pervyj_istochnik_trafika           0
ym_pervyj_perehod_s_sajtov            0
ym_pervaja_poiskovaja_sistema         0
ym_utm_source                         0
ym_utm_content                        0
ym_gorod                              0
ym_vizity                             0
ym_otkazy                             0
ym_glubina_prosmotra                  0
ym_vremja_na_sajte                    0
ym_novye_posetiteli                   0
ym_dnej_mezhdu_vizitami           24833
ym_vernuvshiesja_1_den                0
ym_vernuvshiesja_2_7_dnej             0
ym_dostizhenija_ljuboj_tseli          0
ym_data_pervogo_vizita                0
ym_data_vizita                        0
ym_dostignutaja_tsel                  0
ym_put_polnyj_stranitsy_vhoda         0
ym_put_polnyj_stranitsy_vyhoda        0
ym_mobilnost                          0
dtype: int64

In [11]:
# Корректировка значений
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Не определено', 'ym_pervyj_istochnik_trafika'] = 'undefined'
# ------ Для склейки заменяем пропуски на np.nan
df.loc[df['ym_pervyj_perehod_s_sajtov'] == 'Не определено', 'ym_pervyj_perehod_s_sajtov'] = np.nan
df.loc[df['ym_pervaja_poiskovaja_sistema'] == 'Не определено', 'ym_pervaja_poiskovaja_sistema'] = np.nan
df.loc[df['ym_utm_source'] == 'Не определено', 'ym_utm_source'] = np.nan
df.loc[df['ym_utm_content'] == 'Не определено', 'ym_utm_content'] = np.nan
# ------
df.loc[df['ym_gorod'] == 'Не определено', 'ym_gorod'] = 'undefined'
df.loc[df['ym_otkazy'].isnull(), 'ym_otkazy'] = df['ym_otkazy'].median()
df.loc[df['ym_glubina_prosmotra'].isnull(), 'ym_glubina_prosmotra'] = df['ym_glubina_prosmotra'].median()
df.loc[df['ym_vremja_na_sajte'].isnull(), 'ym_vremja_na_sajte'] = 'undefined'
df.loc[df['ym_dnej_mezhdu_vizitami'].isnull(), 'ym_dnej_mezhdu_vizitami'] = df['ym_dnej_mezhdu_vizitami'].median()
df.loc[df['ym_vernuvshiesja_1_den'].isnull(), 'ym_vernuvshiesja_1_den'] = df['ym_vernuvshiesja_1_den'].median()
df.loc[df['ym_vernuvshiesja_2_7_dnej'].isnull(), 'ym_vernuvshiesja_2_7_dnej'] = df['ym_vernuvshiesja_2_7_dnej'].median()
df.loc[df['ym_data_pervogo_vizita'].isnull(), 'ym_data_pervogo_vizita'] = 'undefined'
df.loc[df['ym_dostignutaja_tsel'].isnull(), 'ym_dostignutaja_tsel'] = 'undefined'
df.loc[df['ym_dostignutaja_tsel'] == 'Не определено', 'ym_dostignutaja_tsel'] = 'undefined'
df.loc[df['ym_put_polnyj_stranitsy_vhoda'].isnull(), 'ym_put_polnyj_stranitsy_vhoda'] = 'undefined'
df.loc[df['ym_put_polnyj_stranitsy_vyhoda'].isnull(), 'ym_put_polnyj_stranitsy_vyhoda'] = 'undefined'
df.loc[df['ym_mobilnost'].isnull(), 'ym_mobilnost'] = 'undefined' # Если не определено
df.loc[df['ym_mobilnost'] == 1.0, 'ym_mobilnost'] = 1
df.loc[df['ym_mobilnost'] == 0.0, 'ym_mobilnost'] = 0

In [12]:
# Приведение типов
df = astype_col(df, ['ym_vizity'], coltype='uint8')
df = astype_col(df, ['ym_novye_posetiteli'], coltype='uint8')
# df = astype_col(df, ['ym_mobilnost'], coltype='uint8')

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27396 entries, 130 to 44624
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ym_clientid                     27396 non-null  object 
 1   ym_pervyj_istochnik_trafika     27396 non-null  object 
 2   ym_pervyj_perehod_s_sajtov      2475 non-null   object 
 3   ym_pervaja_poiskovaja_sistema   1382 non-null   object 
 4   ym_utm_source                   6598 non-null   object 
 5   ym_utm_content                  6501 non-null   object 
 6   ym_gorod                        27396 non-null  object 
 7   ym_vizity                       27396 non-null  uint8  
 8   ym_otkazy                       27396 non-null  float64
 9   ym_glubina_prosmotra            27396 non-null  float64
 10  ym_vremja_na_sajte              27396 non-null  object 
 11  ym_novye_posetiteli             27396 non-null  uint8  
 12  ym_dnej_mezhdu_vizitami       

In [14]:
# Просмотр названий каждого канала
df['ym_pervyj_istochnik_trafika'].unique()

array(['Переходы по рекламе', 'Прямые заходы',
       'Переходы с почтовых рассылок', 'Переходы по ссылкам на сайтах',
       'Переходы из поисковых систем', 'Внутренние переходы',
       'Переходы из социальных сетей',
       'Переходы из рекомендательных систем', 'Переходы из мессенджеров',
       'undefined'], dtype=object)

In [15]:
# Добавляем стоимость клика по каждому каналу
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Прямые заходы', 'ym_cost'] = 0
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы по рекламе', 'ym_cost'] = 15
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы из поисковых систем', 'ym_cost'] = 48.5
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Внутренние переходы', 'ym_cost'] = 0
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы по ссылкам на сайтах', 'ym_cost'] = 0
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы из социальных сетей', 'ym_cost'] = 0
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы с почтовых рассылок', 'ym_cost'] = 2
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы из рекомендательных систем', 'ym_cost'] = 0
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы из мессенджеров', 'ym_cost'] = 0
df.loc[df['ym_pervyj_istochnik_trafika'] == 'undefined', 'ym_cost'] = 0

In [16]:
# Переименовываем категории каналов
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Прямые заходы', 'ym_pervyj_istochnik_trafika'] = 'Browser-Traffic'
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы по рекламе', 'ym_pervyj_istochnik_trafika'] = 'Ad-Traffic'
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы из поисковых систем', 'ym_pervyj_istochnik_trafika'] = 'Search'
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Внутренние переходы', 'ym_pervyj_istochnik_trafika'] = 'Site-Traffic'
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы по ссылкам на сайтах', 'ym_pervyj_istochnik_trafika'] = 'Sites'
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы из социальных сетей', 'ym_pervyj_istochnik_trafika'] = 'Social'
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы с почтовых рассылок', 'ym_pervyj_istochnik_trafika'] = 'Email'
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы из рекомендательных систем', 'ym_pervyj_istochnik_trafika'] = 'RecSys'
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы из мессенджеров', 'ym_pervyj_istochnik_trafika'] = 'Messenger'

In [17]:
df.head(5)

,ym_clientid,ym_pervyj_istochnik_trafika,ym_pervyj_perehod_s_sajtov,ym_pervaja_poiskovaja_sistema,ym_utm_source,ym_utm_content,ym_gorod,ym_vizity,ym_otkazy,ym_glubina_prosmotra,...,ym_vernuvshiesja_1_den,ym_vernuvshiesja_2_7_dnej,ym_dostizhenija_ljuboj_tseli,ym_data_pervogo_vizita,ym_data_vizita,ym_dostignutaja_tsel,ym_put_polnyj_stranitsy_vhoda,ym_put_polnyj_stranitsy_vyhoda,ym_mobilnost,ym_cost
130,1595636421722944149,Ad-Traffic,NaN,NaN,NaN,NaN,Нур-Султан,14,0.571429,1.928571,...,0.928571,0.0,0.0,2020-07-25,2020-07-25,undefined,/#secondPage,/#secondPage,1,15.0
291,159789595254507935,Ad-Traffic,NaN,NaN,NaN,NaN,undefined,9,0.222222,1.555556,...,0.888889,0.0,0.0,2020-08-20,2020-08-20,undefined,/,/,1,15.0
405,1611493557612950817,Ad-Traffic,NaN,NaN,yandex,kontrol_24,undefined,8,0.125000,1.000000,...,0.875000,0.0,8.0,2021-01-24,2021-01-24,Hubex.ru (Ретаргетинг.Все-Страницы) - Посетил ...,/,/,1,15.0
482,1602819673493127299,Ad-Traffic,NaN,NaN,yad,240x400,Валуйки,7,1.000000,1.000000,...,0.857143,0.0,0.0,2020-10-16,2020-10-16,undefined,/,/,1,15.0
525,158655273151274865,Ad-Traffic,NaN,NaN,NaN,NaN,undefined,6,1.000000,1.000000,...,0.833333,0.0,0.0,2020-04-11,2020-04-11,undefined,/,/,1,15.0


In [18]:
# Слияние столбцов
df['ym_source'] = df[df.columns[2:6]].apply(lambda x: ' > '.join(x.dropna().astype(str)), axis=1)

In [19]:
# Выводим имена столцов
get_idx_columns(df.columns)

0 ym_clientid
1 ym_pervyj_istochnik_trafika
2 ym_pervyj_perehod_s_sajtov
3 ym_pervaja_poiskovaja_sistema
4 ym_utm_source
5 ym_utm_content
6 ym_gorod
7 ym_vizity
8 ym_otkazy
9 ym_glubina_prosmotra
10 ym_vremja_na_sajte
11 ym_novye_posetiteli
12 ym_dnej_mezhdu_vizitami
13 ym_vernuvshiesja_1_den
14 ym_vernuvshiesja_2_7_dnej
15 ym_dostizhenija_ljuboj_tseli
16 ym_data_pervogo_vizita
17 ym_data_vizita
18 ym_dostignutaja_tsel
19 ym_put_polnyj_stranitsy_vhoda
20 ym_put_polnyj_stranitsy_vyhoda
21 ym_mobilnost
22 ym_cost
23 ym_source
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


In [20]:
# Новый порядок столбцов
new_order = [0, 1, 23, 2, 3, 4, 5, 6, 22, 16, 17, 7, 8, 9, 10, 11, 12, 13, 14, 15, 18, 19, 20, 21]
df = df[df.columns[new_order]]

In [21]:
df.columns

Index(['ym_clientid', 'ym_pervyj_istochnik_trafika', 'ym_source',
       'ym_pervyj_perehod_s_sajtov', 'ym_pervaja_poiskovaja_sistema',
       'ym_utm_source', 'ym_utm_content', 'ym_gorod', 'ym_cost',
       'ym_data_pervogo_vizita', 'ym_data_vizita', 'ym_vizity', 'ym_otkazy',
       'ym_glubina_prosmotra', 'ym_vremja_na_sajte', 'ym_novye_posetiteli',
       'ym_dnej_mezhdu_vizitami', 'ym_vernuvshiesja_1_den',
       'ym_vernuvshiesja_2_7_dnej', 'ym_dostizhenija_ljuboj_tseli',
       'ym_dostignutaja_tsel', 'ym_put_polnyj_stranitsy_vhoda',
       'ym_put_polnyj_stranitsy_vyhoda', 'ym_mobilnost'],
      dtype='object')

In [22]:
# Корректировка значений
df['ym_source'] = df['ym_source'].str.lower() # Переводим все нижний регистр
df.loc[(df['ym_source'] == 'яндекс') | (df['ym_source'] == 'yandex.ru'), 'ym_source'] = 'yandex'
df.loc[(df['ym_source'] == '') | (df['ym_source'].isnull()), 'ym_source'] = 'undefined' # Убираем Null и пустоты
df = df[~df['ym_source'].str.contains('([0-9]{1,3}[\.]){3}[0-9]{1,3}')] # Убираем ip-адреса
df.loc[df['ym_source'] == 'мобильные%сотрудники', 'ym_source'] = 'undefined'
df.loc[df['ym_source'] == 'старт за 2 дня', 'ym_source'] = 'undefined'
df.loc[df['ym_source'] == 'яндекс > serp', 'ym_source'] = 'yandex'
# -------
df.loc[df['ym_pervyj_perehod_s_sajtov'].isnull(), 'ym_pervyj_perehod_s_sajtov'] = 'undefined'
df.loc[df['ym_pervaja_poiskovaja_sistema'].isnull(), 'ym_pervaja_poiskovaja_sistema'] = 'undefined'
df.loc[df['ym_utm_source'].isnull(), 'ym_utm_source'] = 'undefined'
df.loc[df['ym_utm_content'].isnull(), 'ym_utm_content'] = 'undefined'

In [23]:
df.head(5)

,ym_clientid,ym_pervyj_istochnik_trafika,ym_source,ym_pervyj_perehod_s_sajtov,ym_pervaja_poiskovaja_sistema,ym_utm_source,ym_utm_content,ym_gorod,ym_cost,ym_data_pervogo_vizita,...,ym_vremja_na_sajte,ym_novye_posetiteli,ym_dnej_mezhdu_vizitami,ym_vernuvshiesja_1_den,ym_vernuvshiesja_2_7_dnej,ym_dostizhenija_ljuboj_tseli,ym_dostignutaja_tsel,ym_put_polnyj_stranitsy_vhoda,ym_put_polnyj_stranitsy_vyhoda,ym_mobilnost
130,1595636421722944149,Ad-Traffic,undefined,undefined,undefined,undefined,undefined,Нур-Султан,15.0,2020-07-25,...,00:00:00,1,0.0,0.928571,0.0,0.0,undefined,/#secondPage,/#secondPage,1
291,159789595254507935,Ad-Traffic,undefined,undefined,undefined,undefined,undefined,undefined,15.0,2020-08-20,...,00:00:11,1,0.0,0.888889,0.0,0.0,undefined,/,/,1
405,1611493557612950817,Ad-Traffic,yandex > kontrol_24,undefined,undefined,yandex,kontrol_24,undefined,15.0,2021-01-24,...,00:00:20,1,0.0,0.875000,0.0,8.0,Hubex.ru (Ретаргетинг.Все-Страницы) - Посетил ...,/,/,1
482,1602819673493127299,Ad-Traffic,yad > 240x400,undefined,undefined,yad,240x400,Валуйки,15.0,2020-10-16,...,00:00:00,1,0.0,0.857143,0.0,0.0,undefined,/,/,1
525,158655273151274865,Ad-Traffic,undefined,undefined,undefined,undefined,undefined,undefined,15.0,2020-04-11,...,00:00:00,1,0.0,0.833333,0.0,0.0,undefined,/,/,1


In [24]:
df.describe()

,ym_cost,ym_vizity,ym_otkazy,ym_glubina_prosmotra,ym_novye_posetiteli,ym_dnej_mezhdu_vizitami,ym_vernuvshiesja_1_den,ym_vernuvshiesja_2_7_dnej,ym_dostizhenija_ljuboj_tseli
count,27394.000000,27394.000000,27394.000000,27394.000000,27394.000000,27394.000000,27394.000000,27394.000000,27394.000000
mean,14.351062,1.090531,0.491151,1.251797,0.979375,0.311717,0.041256,0.008736,0.551252
std,9.553335,0.356093,0.493438,0.888671,0.152055,3.611581,0.146748,0.089851,1.105182
min,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,15.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,15.000000,1.000000,0.500000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,15.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000
max,48.500000,14.000000,1.000000,35.000000,5.000000,125.000000,1.000000,1.000000,33.000000


In [25]:
df.shape

(27394, 24)

## Сохранение данных

In [26]:
# Сохраняем в файл
df.to_csv('../data/td_metrika.csv', index=False, encoding='utf-8', sep=';')